In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\David gathara marigi\Downloads\UCI_Credit_Card.csv")

In [3]:
print(df.columns)

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default.payment.next.month'],
      dtype='object')


In [4]:
df[(df['default.payment.next.month'] == 0) & (df['PAY_0'] == -1)].shape[0] / df.shape[0]

0.15773333333333334

In [5]:
train_features = df.drop(['ID', 'default.payment.next.month'], axis=1)
train_target = df['default.payment.next.month']

In [6]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
imputer = SimpleImputer(strategy='mean')
train_features_imputed = imputer.fit_transform(train_features)
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features_imputed)

In [7]:
correlations = pd.DataFrame(train_features, columns=train_features.columns).corrwith(train_target)
print('Feature correlations with target:')
print(correlations.sort_values(ascending=False))

Feature correlations with target:
PAY_0        0.324794
PAY_2        0.263551
PAY_3        0.235253
PAY_4        0.216614
PAY_5        0.204149
PAY_6        0.186866
EDUCATION    0.028006
AGE          0.013890
BILL_AMT6   -0.005372
BILL_AMT5   -0.006760
BILL_AMT4   -0.010156
BILL_AMT3   -0.014076
BILL_AMT2   -0.014193
BILL_AMT1   -0.019644
MARRIAGE    -0.024339
SEX         -0.039961
PAY_AMT6    -0.053183
PAY_AMT5    -0.055124
PAY_AMT3    -0.056250
PAY_AMT4    -0.056827
PAY_AMT2    -0.058579
PAY_AMT1    -0.072929
LIMIT_BAL   -0.153520
dtype: float64


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [9]:
if 'train_features_scaled' not in globals():
	imputer = SimpleImputer(strategy='mean')
	train_features_imputed = imputer.fit_transform(train_features)
	scaler = StandardScaler()
	train_features_scaled = scaler.fit_transform(train_features_imputed)

param_grid = { 'C': [0.01, 0.1, 1, 10, 100] }
lr = LogisticRegression(max_iter=1000)
grid = GridSearchCV(lr, param_grid, cv=5)
grid.fit(train_features_scaled, train_target)
print('Best parameters:', grid.best_params_)
scores = cross_val_score(grid.best_estimator_, train_features_scaled, train_target, cv=5)
print('Cross-validation scores:', scores)
print('Mean CV score:', scores.mean())

Best parameters: {'C': 100}
Cross-validation scores: [0.803      0.80583333 0.8125     0.81616667 0.81183333]
Mean CV score: 0.8098666666666666
